In [24]:
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import tqdm

n = 1000
q = 7/10
steps = 300
lmbda = 0.1
lr = 0.1
N_iters = 100

T = np.random.normal(0, 1/np.sqrt(q * n), n)
T=T/np.sqrt(np.dot(T,T))

def ltr(S, Sigma):
    return np.dot(S - T, Sigma @ (S - T))

def lgen(S):
    return np.dot(S - T, S - T)

def g(S, Sigma):
    return 2 * Sigma @ (S - T)

def computeLosses(iter):
    X = np.random.normal(0, 1, (int(q * n), n))
    Sigma = X.T @ X / (q * n)
    S0 = np.random.normal(0, 1/np.sqrt(q * n), n)
    S = S0
    S2 = S0
    LS_tr = [ltr(S0, Sigma)]
    LS_gen = [lgen(S0)]
    LS_tr_lambda = [ltr(S2, Sigma)]
    LS_gen_lambda = [lgen(S2)]
    for i in range(steps):
        S = S - lr * g(S, Sigma)
        S2 = (S2 - lr * g(S2, Sigma)) / (1 + lr * lmbda)
        LS_tr.append(ltr(S, Sigma))
        LS_gen.append(lgen(S))
        LS_tr_lambda.append(ltr(S2, Sigma))
        LS_gen_lambda.append(lgen(S2))
    return LS_tr, LS_gen, LS_tr_lambda, LS_gen_lambda, np.dot(S0, S0)

allResults = Parallel(n_jobs=-1)(delayed(computeLosses)(i) for i in tqdm.trange(N_iters))

# Initialize accumulators
CumLS_tr = np.zeros(steps + 1)
CumLS_gen = np.zeros(steps + 1)
CumLS_tr_lambda = np.zeros(steps + 1)
CumLS_gen_lambda = np.zeros(steps + 1)
mean_S0_dot = 0.0

# Accumulate results
for res in allResults:
    CumLS_tr += res[0]
    CumLS_gen += res[1]
    CumLS_tr_lambda += res[2]
    CumLS_gen_lambda += res[3]
    mean_S0_dot += res[4]

# Calculate means
MeanLS_tr = CumLS_tr / N_iters
MeanLS_gen = CumLS_gen / N_iters
MeanLS_tr_lambda = CumLS_tr_lambda / N_iters
MeanLS_gen_lambda = CumLS_gen_lambda / N_iters
mean_S0_dot /= N_iters

# Plotting
plt.figure()
plt.plot(MeanLS_tr, label="train")
plt.plot(MeanLS_gen, label="gen")
plt.plot(MeanLS_tr_lambda, label="train_lambda")
plt.plot(MeanLS_gen_lambda, label="gen_lambda")
plt.plot([(1 - q)*(1+mean_S0_dot)] * steps, 'k--')
plt.plot([(1 - q)] * steps, 'r--')
plt.legend()
plt.xlabel("Steps")
plt.ylabel("Mean Loss")
plt.show()


100%|██████████| 100/100 [00:19<00:00,  5.22it/s]


In [19]:
n = 1000
q = 7/10
steps = 300
lmbda = 0.1
lr = 0.1
N_iters = 100

T = np.random.normal(0, 1/np.sqrt(q * n), n)
T=T/np.sqrt(np.dot(T,T))

def ltr(S, Sigma):
    return np.dot(S - T, Sigma @ (S - T))

def lgen(S):
    return np.dot(S - T, S - T)

def g(S, Sigma):
    return 2 * Sigma @ (S - T)

def computeLosses(iter):
    X = np.random.normal(0, 1, (int(q * n), n))
    Sigma = X.T @ X / (q * n)
    S0 = np.random.normal(0, 1/np.sqrt(q * n), n)
    S = S0
    S2 = S0
    LS_tr = [ltr(S0, Sigma)]
    LS_gen = [lgen(S0)]
    LS_tr_lambda = [ltr(S2, Sigma)]
    LS_gen_lambda = [lgen(S2)]
    for i in range(steps):
        S = S - lr * g(S, Sigma)
        S2 = (S2 - lr * g(S2, Sigma)) / (1 + lr * lmbda)
        LS_tr.append(ltr(S, Sigma))
        LS_gen.append(lgen(S))
        LS_tr_lambda.append(ltr(S2, Sigma))
        LS_gen_lambda.append(lgen(S2))
    return LS_tr, LS_gen, LS_tr_lambda, LS_gen_lambda, np.dot(S0, S0)

allResults = Parallel(n_jobs=-1)(delayed(computeLosses)(i) for i in tqdm.trange(N_iters))

# Initialize accumulators
CumLS_tr = np.zeros(steps + 1)
CumLS_gen = np.zeros(steps + 1)
CumLS_tr_lambda = np.zeros(steps + 1)
CumLS_gen_lambda = np.zeros(steps + 1)
mean_S0_dot = 0.0

# Accumulate results
for res in allResults:
    CumLS_tr += res[0]
    CumLS_gen += res[1]
    CumLS_tr_lambda += res[2]
    CumLS_gen_lambda += res[3]
    mean_S0_dot += res[4]

# Calculate means
MeanLS_tr = CumLS_tr / N_iters
MeanLS_gen = CumLS_gen / N_iters
MeanLS_tr_lambda = CumLS_tr_lambda / N_iters
MeanLS_gen_lambda = CumLS_gen_lambda / N_iters
mean_S0_dot /= N_iters

# Plotting
plt.figure()
plt.plot(MeanLS_tr, label="train")
plt.plot(MeanLS_gen, label="gen")
plt.plot(MeanLS_tr_lambda, label="train_lambda")
plt.plot(MeanLS_gen_lambda, label="gen_lambda")
plt.plot([(1 - q)*(1+mean_S0_dot)] * steps, 'k--')
plt.plot([(1 - q)] * steps, 'r--')
plt.legend()
plt.xlabel("Steps")
plt.ylabel("Mean Loss")
plt.show()


2.0000000000000018